In [236]:
from datetime import datetime, timedelta
from itertools import product

import geoip2
import geoip2.database as maxmind
import numpy as np
import pandas as pd
import scipy.stats as sps
from wmfdata import mariadb, hive
from wmfdata.utils import pd_display_all, mediawiki_dt

# Mobile edit samples

In [ ]:
wikis = ["enwiki", "cswiki", "kowiki", "hiwiki", "frwiki"]

mobile_edits = mariadb.multirun("""
select
    database() as wiki,
    rev_id,
    ct_tag
from change_tag
left join revision
on ct_rev_id = rev_id
where
    ct_tag in ("android app edit", "ios app edit", "mobile web edit") and
    rev_timestamp between "201808" and "201811"
""", wikis=wikis)

In [ ]:
mobile_edits.head()

In [ ]:
(mobile_edits
    .groupby(["wiki", "ct_tag"])
    .agg("count")
    .unstack()
)

In [ ]:
sampled_edits = pd.DataFrame(columns=["wiki", "rev_id", "ct_tag"])

for wiki in wikis:
    wiki_edits = mobile_edits.query("wiki == @wiki")
    
    web_edits = wiki_edits.query("ct_tag == 'mobile web edit'").sample(160)
    ios_edits = wiki_edits.query("ct_tag == 'ios app edit'").sample(20)
    android_edits = wiki_edits.query("ct_tag == 'android app edit'").sample(20)
    
    sampled_edits = (sampled_edits
                     .append([web_edits, ios_edits, android_edits])
                     .reset_index(drop=True)
                    )

In [ ]:
# namespaces = {
#     0: "",
#     1: "Talk",
#     2: "User",
#     3: "User talk",
#     4: "Project",
#     5: "Project talk",
#     6: "File",
#     7: "File talk",
#     8: "MediaWiki",
#     9: "MediaWiki talk",
#     10: "Template",
#     11: "Template talk",
#     12: "Help",
#     13: "Help talk",
#     14: "Category",
#     15: "Category talk"
# }

# tags = {
#     "android app edit": "Android app",
#     "ios app edit": "iOS app", 
#     "mobile web edit": "mobile web"
# }

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(sampled_edits
            .replace(to_replace=" edit", value="", regex=True)
            .replace({"android app": "Android app", "ios app": "iOS app"})
           )

# Editor samples

## Load country data

In [151]:
# To-do: make it possible to pass this to 
def pd_bin(bins, other=np.nan):
    """
    Bins must be a dict mapping each bin value to a boolean series giving where it should be applied. 
    All series must be of the same length.
    """
    bin_names = [*bins.keys()]
    masks = [*bins.values()]
    
    lengths = {len(m) for m in masks}
    
    # If the masks do not have equal lengh
    if len(lengths) != 1:
        raise ValueError("Boolean masks must be of equal length.")
        
    # Create the series to hold the result
    length = list(lengths)[0]
    ser = pd.Series(index=range(length))
    
    # Can't use numpy.select since it turns nan values into strings
    for bin_name, mask in bins.items():
        ser[mask] = bin_name
    
    if other is not np.nan:
        ser[ser.isnull()] = other
    
    return ser.astype("category")

In [2]:
# Start included, end excluded
END = datetime(2018, 11, 9)
START = END - timedelta(days=90)

In [3]:
%%time
user_revs = mariadb.run("""
select
    user_name,
    user_email,
    user_editcount as user_edit_count,
    rev_id,
    cuc_ip as rev_ip,
    coalesce(ct_tag = "mobile edit", False) as mobile_rev
from enwiki.revision
left join enwiki.user
on rev_user = user_id
left join enwiki.change_tag
on
    rev_id = ct_rev_id and
    ct_tag = "mobile edit"
left join enwiki.cu_changes
on
    rev_timestamp = cuc_timestamp and
    rev_id = cuc_this_oldid
left join enwiki.user_groups
on
    rev_user = ug_user and
    ug_group = "bot"
left join enwiki.user_former_groups
on
    rev_user = ufg_user and
    ufg_group = "bot"
where
    rev_timestamp between "{start}" and "{end}" and
    rev_user != 0 and
    convert(user_name using utf8) not regexp "bot\\\\b" and
    user_email != "" and
    cuc_ip is not null and
    ufg_group is null and
    ug_group is null
""".format(
    start=mediawiki_dt(START),
    end=mediawiki_dt(END)
))

CPU times: user 1min 31s, sys: 7.89 s, total: 1min 38s
Wall time: 27min 50s


In [13]:
user_revs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7671880 entries, 0 to 7671879
Data columns (total 6 columns):
user_name          object
user_email         object
user_edit_count    int64
rev_id             int64
rev_ip             object
mobile_rev         int64
dtypes: int64(3), object(3)
memory usage: 351.2+ MB


In [6]:
user_revs.to_csv("data/user_revs.tsv", sep="\t", index=False)

In [173]:
%%time 
ip_locator = maxmind.Reader('/usr/share/GeoIP/GeoIP2-Country.mmdb')

def geolocate(ip):
    try:
        return ip_locator.country(ip).country.iso_code
    except geoip2.errors.AddressNotFoundError:
        return "--"

gn_countries = (
    "AD", "AL", "AT", "AX", "BA", "BE", "BG", "CH", "CY", "CZ",
    "DE", "DK", "EE", "ES", "FI", "FO", "FR", "FX", "GB", "GG",
    "GI", "GL", "GR", "HR", "HU", "IE", "IL", "IM", "IS", "IT",
    "JE", "LI", "LU", "LV", "MC", "MD", "ME", "MK", "MT", "NL",
    "NO", "PL", "PT", "RO", "RS", "RU", "SE", "SI", "SJ", "SK",
    "SM", "TR", "VA", "AU", "CA", "HK", "MO", "NZ", "JP", "SG",
    "KR", "TW", "US"
)

def bin_region(ser):
    unk = (ser == "--")
    gn = (ser.isin(gn_countries))
    gs = ~(gn | unk)
    
    return pd.Categorical(
        ser.mask(unk, np.nan).mask(gn, "Global North").mask(gs, "Global South")
    )

user_revs = (
    user_revs
    .assign(rev_country=lambda df: df["rev_ip"].map(geolocate).astype("category"))
    .assign(rev_region=lambda df: bin_region(df["rev_country"]))
)

CPU times: user 7min 12s, sys: 1.04 s, total: 7min 13s
Wall time: 7min 13s


In [176]:
user_revs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7671880 entries, 0 to 7671879
Data columns (total 8 columns):
user_name          object
user_email         object
user_edit_count    int64
rev_id             int64
rev_ip             object
mobile_rev         int64
rev_country        category
rev_region         category
dtypes: category(2), int64(3), object(3)
memory usage: 373.2+ MB


In [177]:
user_revs.to_csv("data/user_revs.tsv", sep="\t", index=False)

In [214]:
%%time
def bin_experience(ser):
    exp = (ser >= 25)
    inexp = ~exp
    return pd.Categorical(
        ser.mask(exp, "experienced").mask(inexp, "inexperienced")
    )

def bin_platform(ser):
    desk = (ser < 0.01)
    mob = (ser > 0.99)
    mix = ~(desk | mob)
    
    return pd.Categorical(
        ser
        .mask(desk, "all desktop")
        .mask(mob, "all mobile")
        .mask(mix, "mixed")
    )

def mode(ser):
    try:
        # Takes the first alphabetically in case of a tie
        return ser.mode()[0]
    except (KeyError, IndexError):
        return np.nan

users = (
    user_revs
    .groupby("user_email") # Group by email rather than user name because some email addresses are duplicated
    .agg({
        "user_name": lambda s: mode(s),
        "user_edit_count": "max",
        "rev_id": len,
        "mobile_rev": "mean",
        "rev_region": lambda s: mode(s) 
    })
    .rename(columns={
        "mobile_rev": "mobile_prop",
        "user_name": "FirstName",
        "user_edit_count": "edit_count",
        "rev_region": "region",
        "rev_id": "edits_in_period"
    })
    .assign(
        experience=lambda df: bin_experience(df["edit_count"]),
        platform=lambda df: bin_platform(df["mobile_prop"])
    )
)

CPU times: user 2min 6s, sys: 1.61 s, total: 2min 7s
Wall time: 2min 6s


In [224]:
frame = users.query("edits_in_period >= 5")

In [226]:
pd.pivot_table(
    frame,
    index=["region", "experience"],
    columns="platform",
    values="FirstName",
    aggfunc=len
).stack()

region        experience     platform   
Global North  experienced    all desktop    26304
                             all mobile       552
                             mixed           4657
              inexperienced  all desktop    15325
                             all mobile      1650
                             mixed           1361
Global South  experienced    all desktop     4705
                             all mobile       416
                             mixed           1742
              inexperienced  all desktop     4587
                             all mobile      1424
                             mixed           1007
dtype: int64

In [ ]:
frame = (
    frame
    .rename_axis("Email")
    .rename(columns={
        "mobile_prop": "mobile_proportion",
        "edit_count": "total_edit_count",
    })
)

cols = frame.columns.tolist()

cols = [cols[-3]] + cols[:4] + cols[5:]

frame = frame[cols]

frame.head()

In [252]:
query_string = "experience == '{exp}' & platform == '{plat}' & region == '{reg}'"

experience = ("inexperienced", "experienced")
platform = ("all desktop", "all mobile", "mixed")
region = ("Global South", "Global North")
platform_x_region = [x for x in product(platform, region)]

random_seed = 434194029

for exp in experience:
    samples = []
    for plat, reg in platform_x_region:
        formatted_query = query_string.format(exp=exp, plat=plat, reg=reg)
        sample = frame.query(formatted_query).sample(210, random_state=random_seed)
        samples.append(sample)
    
    contact_list = pd.concat(samples)
    filename = "data/mobile-survey-" + exp + ".csv"
    contact_list.to_csv(filename)